# Lecture 6

Data Science, Fall 2023



A demonstration of exploratory data analysis to accompany Lecture 6 (originally planned for Lecture 5).

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns',7)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


These options are used to customize the display of pandas DataFrames for better readability and precision when working with data. Here's why each option is helpful:

# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

<br/>

---

# CSV and Nice Field Names
Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`.

We can then explore the CSV (which is a text file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

1, 2. Let's start with the first two so we really solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.



4. Finally, let's see the tried-and-true Data Science approach: pandas.

In [17]:
# Read tuberculosis data

tuber_data = pd.read_csv("H:\Machine Learning\EDA\cdc_tuberculosis.csv")
tuber_data.head(2)


,Unnamed: 0,No. of TB cases,Unnamed: 2,Unnamed: 3,TB incidence,Unnamed: 5,Unnamed: 6
0,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,Total,"8,900","7,173","7,860",2.71,2.16,2.37


Wait, what's up with the "Unnamed" column names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [18]:
# Answer Here

tuber_data = pd.read_csv("H:\Machine Learning\EDA\cdc_tuberculosis.csv",header=1)
tuber_data.head(2)

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83


Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()` [documentation]

In [20]:
# Rename column List
tuber_data.columns = ['States','TB_cases(2019)','TB_cases(2020)','TB_cases(2021)','TB_inci(2019)','TB_inci(2020)','TB_inci(2021)']

In [21]:
tuber_data.head(2)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021)
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83


In [23]:
cols = tuber_data.columns
cols

Index(['States', 'TB_cases(2019)', 'TB_cases(2020)', 'TB_cases(2021)',
       'TB_inci(2019)', 'TB_inci(2020)', 'TB_inci(2021)'],
      dtype='object')

<br/><br/>

---

# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.

<br/>

Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [24]:
# Apply sum to tb DataFrame

tuber_data["Total_Cases"] = tuber_data[['TB_cases(2019)','TB_cases(2020)','TB_cases(2021)']].sum(axis=1)
tuber_data["Total_incidence"] = tuber_data[['TB_inci(2019)','TB_inci(2020)','TB_inci(2021)']].sum(axis=1)


In [25]:
tuber_data.head()

,States,TB_cases(2019),TB_cases(2020),...,TB_inci(2021),Total_Cases,Total_incidence
0,Total,"8,900","7,173",...,2.37,"8,9007,1737,860",7.24
1,Alabama,87,72,...,1.83,877292,5.03
2,Alaska,58,58,...,7.92,585858,23.75
3,Arizona,183,136,...,1.77,183136129,6.17
4,Arkansas,64,59,...,2.28,645969,6.36


<br/>

Whoa, what's going on? Check out the column types:

In [26]:
# Find datatype for each column
tuber_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   States           52 non-null     object 
 1   TB_cases(2019)   52 non-null     object 
 2   TB_cases(2020)   52 non-null     object 
 3   TB_cases(2021)   52 non-null     object 
 4   TB_inci(2019)    52 non-null     float64
 5   TB_inci(2020)    52 non-null     float64
 6   TB_inci(2021)    52 non-null     float64
 7   Total_Cases      52 non-null     object 
 8   Total_incidence  52 non-null     float64
dtypes: float64(4), object(5)
memory usage: 3.8+ KB


<br/>

Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

<br/>

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [29]:
# improve readability: chaining method calls with outer parentheses/line breaks

tuber_data = pd.read_csv("H:\Machine Learning\EDA\cdc_tuberculosis.csv",thousands=',',header=1)

tuber_data.columns = cols

tuber_data.head(2)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021)
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83


In [30]:
# now apply sum
# Answer Here


tuber_data["Total_Cases"] = tuber_data[['TB_cases(2019)','TB_cases(2020)','TB_cases(2021)']].sum(axis=1)
tuber_data["Total_incidence"] = tuber_data[['TB_inci(2019)','TB_inci(2020)','TB_inci(2021)']].sum(axis=1)





In [33]:
tuber_data.head()

,States,TB_cases(2019),TB_cases(2020),...,TB_inci(2021),Total_Cases,Total_incidence
0,Total,8900,7173,...,2.37,23933,7.24
1,Alabama,87,72,...,1.83,251,5.03
2,Alaska,58,58,...,7.92,174,23.75
3,Arizona,183,136,...,1.77,448,6.17
4,Arkansas,64,59,...,2.28,192,6.36


The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

In [34]:
# Answer Here
tuber_data.drop(0,axis=0,inplace=True)
tuber_data.head()

,States,TB_cases(2019),TB_cases(2020),...,TB_inci(2021),Total_Cases,Total_incidence
1,Alabama,87,72,...,1.83,251,5.03
2,Alaska,58,58,...,7.92,174,23.75
3,Arizona,183,136,...,1.77,448,6.17
4,Arkansas,64,59,...,2.28,192,6.36
5,California,2111,1706,...,4.46,5567,14.13


What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [41]:
# Load 2010s census data
# Answer Here

cen_data = pd.read_csv(r"H:\Machine Learning\EDA\nst-est2019-01.csv",header=3,thousands=",")
cen_data.head()


,Unnamed: 0,Census,Estimates Base,...,2017,2018,2019
0,United States,308745538.00,308758105.00,...,324985539.00,326687501.00,328239523.00
1,Northeast,55317240.00,55318443.00,...,56059240.00,56046620.00,55982803.00
2,Midwest,66927001.00,66929725.00,...,68126781.00,68236628.00,68329004.00
3,South,114555744.00,114563030.00,...,123542189.00,124569433.00,125580448.00
4,West,71945553.00,71946907.00,...,77257329.00,77834820.00,78347268.00


In [77]:
cen_data2 = pd.read_csv(r"H:\Machine Learning\EDA\NST-EST2022-POP.csv",header=3,thousands=",")
cen_data2.head()

cen_data2.rename(columns={"Unnamed: 0": "Geographic Area"},inplace=True)

# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns, use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [ ]:

# with pd.option_context('display.min_rows', 30): # shows more rows
#     display(census_2010s_df)

cen_data = (
    cen_data
    .drop(columns=["Estimates Base"])
    .rename(columns={"Unnamed: 0": "Geographic Area"})
    .convert_dtypes()                
    .dropna()                   
)


In [80]:
cen_data2.dropna(inplace=True)


In [82]:
# census 2020s data
cen_data2.head()

,Geographic Area,Unnamed: 1,2020,2021,2022
0,United States,331449520.00,331511512.00,332031554.00,333287557.00
1,Northeast,57609156.00,57448898.00,57259257.00,57040406.00
2,Midwest,68985537.00,68961043.00,68836505.00,68787595.00
3,South,126266262.00,126450613.00,127346029.00,128716192.00
4,West,78588565.00,78650958.00,78589763.00,78743364.00


In [46]:
cen_data["Geographic Area"].str.len().head(2)

0    13
1     9
Name: Geographic Area, dtype: Int64

In [85]:
cen_data["Geographic Area"] = cen_data["Geographic Area"].str.strip('.')
cen_data2["Geographic Area"] = cen_data2["Geographic Area"].str.strip('.')

In [86]:
# perform EDA Here

print(cen_data.columns)
print(tuber_data.columns)
print(cen_data2.columns)

Index(['Geographic Area', 'Census', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
Index(['States', 'TB_cases(2019)', 'TB_cases(2020)', 'TB_cases(2021)',
       'TB_inci(2019)', 'TB_inci(2020)', 'TB_inci(2021)', 'Total_Cases',
       'Total_incidence'],
      dtype='object')
Index(['Geographic Area', 'Unnamed: 1', '2020', '2021', '2022'], dtype='object')


<br/><br/>

---

# Join Data (Merge DataFrames)

Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [87]:
# merge TB dataframe with two US census dataframes
merged = tuber_data.merge(right=cen_data,
           left_on="States", right_on="Geographic Area").merge(right=cen_data2,
           left_on="States", right_on="Geographic Area")

In [88]:
pd.set_option('display.max_columns',33)
merged

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence,Geographic Area_x,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Geographic Area_y,Unnamed: 1,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,251,5.03,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5024356.00,5031362.00,5049846.00,5074296.00
1,Alaska,58,58,58,7.91,7.92,7.92,174,23.75,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,Alaska,733378.00,732923.00,734182.00,733583.00
2,Arizona,183,136,129,2.51,1.89,1.77,448,6.17,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,Arizona,7151507.00,7179943.00,7264877.00,7359197.00
3,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,Arkansas,3011555.00,3014195.00,3028122.00,3045637.00
4,California,2111,1706,1750,5.35,4.32,4.46,5567,14.13,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,California,39538245.00,39501653.00,39142991.00,39029342.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,521,6.05,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,Virginia,8631384.00,8636471.00,8657365.00,8683619.00
47,Washington,221,163,199,2.90,2.11,2.57,583,7.58,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,Washington,7705247.00,7724031.00,7740745.00,7785786.00
48,West Virginia,9,13,7,0.50,0.73,0.39,29,1.62,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,West Virginia,1793755.00,1791420.00,1785526.00,1775156.00
49,Wisconsin,51,35,66,0.88,0.59,1.12,152,2.59,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,Wisconsin,5893725.00,5896271.00,5880101.00,5892539.00


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.

In [91]:

# try merging again, but cleaner this time
merged = (
    tuber_data
    .merge(right=cen_data[["Geographic Area", "2019"]],
           left_on="States", right_on="Geographic Area").drop(columns="Geographic Area").merge(right=cen_data2,
           left_on="States", right_on="Geographic Area").drop(columns=["Geographic Area","Unnamed: 1"])
)
merged.head()

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence,2019,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,251,5.03,4903185,5031362.00,5049846.00,5074296.00
1,Alaska,58,58,58,7.91,7.92,7.92,174,23.75,731545,732923.00,734182.00,733583.00
2,Arizona,183,136,129,2.51,1.89,1.77,448,6.17,7278717,7179943.00,7264877.00,7359197.00
3,Arkansas,64,59,69,2.12,1.96,2.28,192,6.36,3017804,3014195.00,3028122.00,3045637.00
4,California,2111,1706,1750,5.35,4.32,4.46,5567,14.13,39512223,39501653.00,39142991.00,39029342.00


In [92]:
merged.drop(columns=["Total_Cases",	"Total_incidence"],inplace=True)


## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

In [93]:
merged["recompute incidence 2019"] = merged["TB_inci(2019)"]/merged['TB_inci(2019)']*100000

In [64]:
merged.head(2)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),Total_Cases,Total_incidence,2019,recompute incidence 2019
0,Alabama,87,72,92,1.77,1.43,1.83,251,5.03,4903185,0.04
1,Alaska,58,58,58,7.91,7.92,7.92,174,23.75,731545,1.08


Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [94]:
# recompute incidence for all years

years = ["TB_cases(2019)","TB_cases(2020)","TB_cases(2021)"]
for year in years:
    merged[f"recompute incidence {year}"] = merged[f"{year}"]/merged['2019']*100000
merged.head(5)

,States,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),2019,2020,2021,2022,recompute incidence 2019,recompute incidence TB_cases(2019),recompute incidence TB_cases(2020),recompute incidence TB_cases(2021)
0,Alabama,87,72,92,1.77,1.43,1.83,4903185,5031362.00,5049846.00,5074296.00,0.04,1.77,1.47,1.88
1,Alaska,58,58,58,7.91,7.92,7.92,731545,732923.00,734182.00,733583.00,1.08,7.93,7.93,7.93
2,Arizona,183,136,129,2.51,1.89,1.77,7278717,7179943.00,7264877.00,7359197.00,0.03,2.51,1.87,1.77
3,Arkansas,64,59,69,2.12,1.96,2.28,3017804,3014195.00,3028122.00,3045637.00,0.07,2.12,1.96,2.29
4,California,2111,1706,1750,5.35,4.32,4.46,39512223,39501653.00,39142991.00,39029342.00,0.01,5.34,4.32,4.43


These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!

In [95]:
merged.describe()

,TB_cases(2019),TB_cases(2020),TB_cases(2021),TB_inci(2019),TB_inci(2020),TB_inci(2021),2019,2020,2021,2022,recompute incidence 2019,recompute incidence TB_cases(2019),recompute incidence TB_cases(2020),recompute incidence TB_cases(2021)
count,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00
mean,174.51,140.65,154.12,2.10,1.78,1.97,6436069.08,6500225.73,6510422.63,6535050.14,0.09,2.10,1.80,1.99
std,341.74,271.06,286.78,1.50,1.34,1.48,7360660.47,7408168.46,7394300.08,7423508.19,0.17,1.50,1.35,1.49
min,1.00,0.00,2.00,0.17,0.00,0.21,578759.00,577605.00,579483.00,581381.00,0.01,0.17,0.00,0.22
25%,25.50,29.00,23.00,1.29,1.21,1.23,1789606.00,1820311.00,1844920.00,1857094.50,0.02,1.30,1.23,1.25
50%,70.00,67.00,69.00,1.80,1.52,1.70,4467673.00,4507445.00,4506589.00,4512310.00,0.04,1.81,1.52,1.71
75%,180.50,139.00,150.00,2.58,1.99,2.22,7446805.00,7451987.00,7502811.00,7572491.50,0.06,2.58,2.02,2.24
max,2111.00,1706.00,1750.00,7.91,7.92,7.92,39512223.00,39501653.00,39142991.00,39029342.00,1.08,7.93,7.93,7.93
